In [ ]:
# Libraries

import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim 

# Read information 
!pip install --upgrade xlrd
!pip install --upgrade openpyxl
!gdown https://drive.google.com/uc?id=1KeAiGZHqGqCcYEUQfpGAyO8eAvN4Wmve
!gdown https://drive.google.com/uc?id=1GLaiLdQdI7MKPMqWy41OI7oITz_eAfhU
!gdown https://drive.google.com/uc?id=1Nb3_iXEvEYMGBxpiTpIA0w_mutq6a-Pu
!gdown https://drive.google.com/uc?id=1p-7gsX9GvGSu7XptfpY-Uu69x-NNJcQd
!gdown https://drive.google.com/uc?id=1d3IH4JVSrV_1rsBcmoFM7AZ_6z2AcdSz



In [ ]:
asesinatos = pd.read_excel('/content/AsesinatosSelectivos1981-2012.xlsx')
periodistas = pd.read_csv('/content/PeriodistasAsesinados.csv')
violencia = pd.read_csv('/content/Violencia.csv')

In [ ]:
# creación de BD Final 
columns = ['Departamento','Municipio','Fuente','Victima','Agente','Año','Tipo_agresion']
df_final = pd.DataFrame(columns = columns)

# Data Cleaning
------

DataFrames usados:

* asesinatos
* periodistas
* violencia

In [ ]:
# Eliminación de columnas basura para cada dataset

# Estructura Dataset ---> ['Departamento','Municipio','Fuente','Victima','Agente','Año','Tipo_agresion']

# Dataset Asesinatos

#asesinatos.info()
asesinatos_f = asesinatos.drop(columns = ['Día','Mes','Lugar de Ocurrencia','Tipo de Implicado (2)','Unnamed: 10'])
asesinatos_final = pd.DataFrame().assign(Departamento = asesinatos_f['Departamento'],Municipio = asesinatos_f['Municipio'],
                                        Agente = asesinatos_f['Tipo de Implicado (1)'],Victima = asesinatos_f['Nº Víctimas'],
                                        Fuente = asesinatos_f['Fuente'],Año = asesinatos_f['Año'],Tipo_agresion = np.nan)


asesinatos_final['Año'] = asesinatos_final['Año'].fillna(0)
asesinatos_final['Año'] = asesinatos_final['Año'].astype(int)


In [ ]:
# Dataset Periodistas

periodistas_f = periodistas.drop(columns = ['X','Y','OBJECTID','Codigo_Municipio','Descripcion','Fecha_Actualizacion','Total_asesinatos_00_15','Total_asesinatos_75_15'])
list_asesinatosPeri = []
list_años = []

periodistas_f = periodistas_f.assign(Victimas = 0)

for i in range(len(periodistas['Total_asesinatos_00_15'])):
  periodistas_f['Victimas'][i] = (periodistas['Total_asesinatos_00_15'][i]+periodistas['Total_asesinatos_75_15'][i])
  año = periodistas_f['Periodo'][i].split('-')
  list_años.append(año[1])

periodistas_final = pd.DataFrame().assign(Departamento = periodistas_f['Nombre_Departamento'],Municipio = periodistas_f['Nombre_Municipio'],
                                          Agente = np.nan,Victima = periodistas_f['Victimas'],Fuente = periodistas_f['Fuente'],Año = list_años,
                                          Tipo_agresion = np.nan)
periodistas_final.head()

In [ ]:
# Dataset Victimas

violencia_f = violencia.drop(columns = ['ID Caso','ID Caso Relacionado','Mes','Día','Código DANE de Municipio','Región',
                                        'Descripción Presunto Responsable','latitud-longitud'])
tipo_violenciaList = []

for index in violencia.index: 
  tipo = 'NaN'
  if violencia['Abandono o Despojo Forzado de Tierras'][index] == 1: 
    tipo = 'Abandono o Despojo Forzado de Tierras'
  elif violencia['Amenaza o Intimidación'][index] == 1:
    tipo = 'Amenaza o Intimidación'
  elif violencia['Ataque Contra Misión Médica'][index] == 1:
    tipo = 'Ataque Contra Misión Médica'
  elif violencia['Confinamiento o Restricción a la Movilidad'][index] == 1:
    tipo = 'Confinamiento o Restricción a la Movilidad'
  elif violencia['Desplazamiento Forzado'][index] == 1:
    tipo = 'Desplazamiento Forzado'
  elif violencia['Extorsión'][index] == 1 : 
    tipo = 'Extorsión'
  elif violencia['Lesionados Civiles'][index] == 1 :
    tipo = 'Lesionados Civiles'
  elif violencia['Pillaje'][index] == 1 :
    tipo = 'Pillaje'
  elif violencia['Tortura'][index] == 1 :
    tipo = 'Tortura'
  elif violencia['Violencia Basada en Género'][index] == 1 :
    tipo = 'Violencia Basada en Género'
  elif violencia['Otro Hecho Simultáneo'][index] == 1 :
    tipo = 'Otro Hecho Simultáneo'
  tipo_violenciaList.append(tipo)

violencia_final = pd.DataFrame().assign(Departamento = violencia_f['Departamento'], Municipio =  violencia_f['Municipio'],
                                        Agente = violencia_f['Presunto Responsable'],Victima = violencia_f['Total de Víctimas del Caso'],
                                        Fuente = 'CNMH',Año = violencia_f['Año'],Tipo_agresion =  tipo_violenciaList)  
violencia_final['Victima'].head()

In [ ]:
# Agregar datos finales a la matriz resultante
#print(pd.DataFrame(periodistas_final.groupby('Victima')))
df_final = df_final.append(periodistas_final)
df_final = df_final.append(asesinatos_final)
df_final = df_final.append(violencia_final)

print(pd.DataFrame(df_final.groupby('Victima'))[0])


In [ ]:
# Correción de tipos de datos para cada columna
columnas = list(df_final.columns)
columnas.remove('Año')
columnas.remove('Victima')

for columna in columnas:
  df_final[columna] = df_final[columna].str.upper().str.title() 

nan_index = list((df_final[df_final['Departamento'].isnull()]).index)
df_final = df_final.drop(nan_index,axis=0)

pd.DataFrame(df_final.groupby(df_final['Año']))
print(pd.DataFrame(df_final.groupby('Victima'))[0])


In [ ]:
df_final.info()
pd.DataFrame(df_final.groupby('Victima'))

In [ ]:
sum = 0
lista_A = [' Antioquia','Antioquia ']
for index in df_final.index:
  if str(df_final['Departamento'][index]) ==' Antioquia' or str(df_final['Departamento'][index]) == 'Antioquia ' or str(df_final['Departamento'][index]) == 'Antioquia': 
    df_final['Departamento'][index] = 'Antioquia'
  
  elif str(df_final['Departamento'][index]) == 'Archipielago De San Andres, Providencia Y Santa Catalina':
    df_final['Departamento'][index] = 'Archipielago De San Andres Providencia Y Santa Catalina'

  elif str(df_final['Departamento'][index]) == 'Atlántico':
    df_final['Departamento'][index] = 'Atlanticsso'

  elif str(df_final['Departamento'][index]) == 'Bolívar':
    df_final['Departamento'][index] = 'Bolivar'

  elif str(df_final['Departamento'][index]) == 'Boyacá':
    df_final['Departamento'][index] = 'Boyaca'

  elif str(df_final['Departamento'][index]) == 'Caldas ':
    df_final['Departamento'][index] = 'Caldas'


In [ ]:
df_final

In [ ]:

df_final.Departamento = df_final.Departamento.replace({"Archipielago De San Andres, Providencia Y Santa Catalina": "Archipielago De San Andres Providencia Y Santa Catalina",
                                                       "Antioquia ": "Antioquia","Atlántico":"Atlantico","Bolívar":"Bolivar","Boyacá":"Boyaca",
                                                       "Caldas ":"Caldas","Caquetá":"Caqueta","Cauca ":"Cauca","Chocó":"Choco","Chocó ":"Choco",
                                                       "Cundinamarca ":"Cundinamarca","César ":"César","César":"Cesar","Córdoba":"Cordoba",
                                                       "Distrito Capital":"Bogota, D. C.","Exterior ":"Exterior","Guaviare ":"Guaviare","Córdoba ":"Cordoba",
                                                       "Huila ":"Huila","la Guajira ":"La Guajira","Magdalena ":"Magdalena","Meta ":"Meta","Nariño ":"Nariño",
                                                       "Norte De Santander ":"Norte De Santander","La Guajira ":"La Guajira","Putumayo ":"Putumayo",
                                                       "Quindío":"Quindio","Quindío ":"Quindio","Santafe De Bogota D.C":"Bogota, D. C.",
                                                       "Santander ":"Santander","Sucre ":"Sucre","Tolima ":"Tolima","Vaupés":"Vaupes",
                                                       "Panamá ":"Panamá"," Antioquia":"Antioquia"," Arauca":"Arauca"," Santander":"Santander",
                                                       "Atlanticsso":"Atlantico","Bogota, D. C.":"Bogota","San Andrés Isla":"San Andres",
                                                       "Archipielago De San Andres Providencia Y Santa Catalina":"San Andres","Archipielago De San Andres Providencia Y Santa Catalina":"San Andres"})
# ELiminar panama, Sin Informacion,Venezuela, Venezuela Exterior)
delete_index = []
delete_index.append(list(df_final[df_final['Departamento'] == 'Panamá'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Sin Informacion'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Venezuela'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Venezuela (Exterior)'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Panamá  '].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Brasil'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Exterior'].index))
delete_index.append(list(df_final[df_final['Departamento'] == 'Ecuador'].index))
delete_index.append(list(df_final[df_final['Departamento'] == ' '].index))
 
for i in delete_index:
    df_final = df_final.drop(i,axis=0)


In [ ]:
df_final
copia_df = df_final

#df_final['Victima'] = copia_df['Victima'].fillna(0)
#df_final['Victima'][df_final[df_final['Victima'] == '´1'].index[0]] = 1
pd.DataFrame(df_final.groupby(df_final.Victima))

In [ ]:
links=[
    '6kxs1V_XTL-f',
    'Gj-Wre1mUzow',
    '647PGv2LVxL0',
    'jDpU1kHTVxL3',
    'MElpBJD9VxL5',
    '0PW5lnoVxL6',
    'WelHHAKMVxL8',
    'hRh1RHVOVxL9',
    'C--FWhkFVxUA',
    'Nw4k6xRuVxUD',
    'DCoa0FgBVxUE',
    'GQQn2O0KVxUG',
    'jtpooRbqVxUI',
    'Tl_pNWU1VxUK',
    'OgNbHA1jVxfX',
    'pk2lqE1lVxfZ',
    'gqc8a4ZiVxfa',
    'uo6hqDFIVxfc',
    'P39dehZxVxfd',
    'jS5DXQGBVxfe',
    'OAAfvS7UVw12',
    '8jLpJHoNVw16',
    'Gus9oi3LVw17',
    'zvv--3vTVw19',
    'MCG0D8WIVw1_',
    '0By7KvzsVw2A',
    'QMibGCBnVFeX',
    'Tkt5ILwyVvRU',
    '81WinV6lVvRZ',
    '8zDpFABUVvRe',
    'mg5VwJ1TgYXj',
    'q5WGTGaAgYXn',
    '8FPiqwR4gZIF',
    '8FPiqwR4gZIF',
    '8FPiqwR4gZIF',
    '8FPiqwR4gZIF'
]
print(len(links))

# Creación JSON 
------


In [ ]:

# Estructura Dataset ---> ['Departamento','Municipio','Fuente','Victima','Agente','Año','Tipo_agresion']
pd.DataFrame(df_final.groupby('Victima'))

lista_dep = list(pd.DataFrame(df_final.groupby('Departamento'))[0])
print(len(lista_dep))
data = {}
max_vict = []
maxima_victima = 6404
cont = 0 
for i in lista_dep:
  loc = Nominatim(user_agent="GetLoc") 
  getLoc = loc.geocode(i)
  print(i,getLoc.latitude)
  print(i,getLoc.longitude)
  
  lista_temp = []

  # Victima : value, tipo_agresion : list, agente : list, fuente : list
  depSerie = df_final.groupby('Departamento').get_group(i)
  victima_temp = str(depSerie.Victima.count())
  max_vict.append(victima_temp)
  fuente_temp = list(pd.DataFrame(depSerie.groupby('Fuente'))[0])
  
  try:
    tipo_temp = list(pd.DataFrame(depSerie.groupby('Tipo_agresion'))[0])
  except:
    tipo_temp = list("No hay registro")
  probabilidad = (int(victima_temp)/maxima_victima)*100
  
  dict_temp = {'Latitud':getLoc.latitude,'Longitud':getLoc.longitude,'Victima':victima_temp,
               'Fuente':fuente_temp,'Tipo_agresion':tipo_temp,'Rango':round((int(victima_temp)/maxima_victima)*10),
               'Probabilidad': str(round(probabilidad,3))+'%','Link': 'https://colab.research.google.com/drive/1FL32nTDi76p_aRULSAkQrgwfN-IMpqdE#scrollTo='+links[cont]}

  lista_temp.append(dict_temp)
  data[i] = lista_temp
  cont += 1
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

df_final.head()
df_final.to_csv('BaseDatosGrupoCanela.csv')

In [ ]:
maxi = 0
for i in max_vict: 
  maxi = max(int(i),maxi)
for i in max_vict:
  print(round((int(i)/maxi)*10))
print(maxi)
print(max_vict)

In [ ]:
df_final.info()
violencia.info()